In [ ]:
# QuantBook Analysis Tool 
# For more information see [https://www.quantconnect.com/docs/v2/our-platform/research/getting-started]
qb = QuantBook()
spy = qb.AddEquity("SPY")
history = qb.History(qb.Securities.Keys, 360, Resolution.Daily)

# Indicator Analysis
bbdf = qb.Indicator(BollingerBands(30, 2), spy.Symbol, 360, Resolution.Daily)
bbdf.drop('standarddeviation', axis=1).plot()

In [ ]:
from AlgorithmImports import *
import random
from datetime import datetime, timedelta

def fetch_random_slices(symbol, frontier_date, ending_date, interval=Resolution.Minute, 
                        slice_length=timedelta(days=1), n_slices=5):
    """
    Picks independent slices of stock movement. Each slice is:
    random start time → start + slice_length.
    
    Parameters
    ----------
    symbol : str
        Ticker symbol (e.g., "AAPL").
    frontier_date : datetime
        Earliest possible start date.
    ending_date : datetime
        Latest possible end date.
    interval : Resolution
        Data granularity (Minute, Hour, Daily).
    slice_length : timedelta
        How long each slice should be (e.g., 1 day, 3 hours).
    n_slices : int
        Number of random slices to return.
    
    Returns
    -------
    list of dicts { "start_date", "start_price", "end_date", "end_price" }
    """
    
    # Load all history once
    history = qb.History([symbol], frontier_date, ending_date, interval)
    if history.empty:
        return []
    
    data = history["close"].reset_index()
    
    results = []
    for _ in range(n_slices):
        # Pick random start time ensuring we can go slice_length forward
        latest_start = ending_date - slice_length
        if frontier_date >= latest_start:
            break  # Not enough room for slices
        
        rand_start = frontier_date + timedelta(
            seconds=random.randint(0, int((latest_start - frontier_date).total_seconds()))
        )
        rand_end = rand_start + slice_length
        
        # Find nearest available prices
        start_row = data.loc[data["time"] >= rand_start].head(1)
        end_row   = data.loc[data["time"] >= rand_end].head(1)
        
        if not start_row.empty and not end_row.empty:
            results.append({
                "start_date": start_row["time"].iloc[0],
                "start_price": float(start_row["close"].iloc[0]),
                "end_date": end_row["time"].iloc[0],
                "end_price": float(end_row["close"].iloc[0])
            })
    
    return results


# Example usage
qb = QuantBook()
symbol = qb.AddEquity("AAPL").Symbol

frontier_date = datetime(2015, 1, 1)
ending_date   = datetime(2025, 9, 15)

slices = fetch_random_slices(
    symbol, frontier_date, ending_date,
    interval=Resolution.Minute,
    slice_length=timedelta(days=3),   # 3-day window
    n_slices=5
)

for s in slices:
    print(s)


In [ ]:
from AlgorithmImports import *
from datetime import datetime, timedelta

def fetch_consecutive_slices(symbol, frontier_date, ending_date, interval=Resolution.Minute, 
                             slice_length=timedelta(days=1), max_slices=None):
    """
    Fetches consecutive, non-overlapping slices of stock movement.
    Each slice is frontier_date → frontier_date+slice_length, then continues until ending_date.
    
    Parameters
    ----------
    symbol : str
        Ticker symbol (e.g., "AAPL").
    frontier_date : datetime
        Earliest possible start date.
    ending_date : datetime
        Latest possible end date.
    interval : Resolution
        Data granularity (Minute, Hour, Daily).
    slice_length : timedelta
        Length of each slice (e.g., 1 day, 3 hours).
    max_slices : int, optional
        Maximum number of slices to return. Default = all possible slices.
    
    Returns
    -------
    list of dicts { "start_date", "start_price", "end_date", "end_price" }
    """
    
    # Load history once
    history = qb.History([symbol], frontier_date, ending_date, interval)
    if history.empty:
        return []
    
    data = history["close"].reset_index()
    
    results = []
    current_start = frontier_date
    n_slices = 0
    
    while current_start + slice_length <= ending_date:
        current_end = current_start + slice_length
        
        # Find nearest available prices
        start_row = data.loc[data["time"] >= current_start].head(1)
        end_row   = data.loc[data["time"] >= current_end].head(1)
        
        if not start_row.empty and not end_row.empty:
            results.append({
                "start_date": start_row["time"].iloc[0],
                "start_price": float(start_row["close"].iloc[0]),
                "end_date": end_row["time"].iloc[0],
                "end_price": float(end_row["close"].iloc[0])
            })
            n_slices += 1
        
        # Stop if we've hit the slice cap
        if max_slices is not None and n_slices >= max_slices:
            break
        
        # Advance to next slice
        current_start = current_end
    
    return results

In [ ]:
import pandas as pd
import numpy as np

def slices_to_extended_dataframe(slices, time_format="%Y-%m-%d|%H:%M"):
    """
    Converts slice outputs into an extended DataFrame with:
    - linked date ranges
    - raw price change
    - % change
    - log returns (%)
    - cumulative % return (compounded)
    - cumulative log return (%)
    - cumulative return from logs (%)
    
    Parameters
    ----------
    slices : list of dicts
        Output from fetch_random_slices or fetch_consecutive_slices.
    time_format : str
        How to format the timestamps in the linked range.
    
    Returns
    -------
    pd.DataFrame
        Columns: 
        [date_range, start_date, end_date, raw_change, pct_change, 
         log_return, cum_return, cum_log_return, cum_return_from_logs]
    """
    if not slices:
        return pd.DataFrame(columns=[
            "date_range", "start_date", "end_date", 
            "raw_change", "pct_change", "log_return", 
            "cum_return", "cum_log_return", "cum_return_from_logs"
        ])
    
    rows = []
    for s in slices:
        start_date = s["start_date"]
        end_date   = s["end_date"]
        start_price = s["start_price"]
        end_price   = s["end_price"]
        
        # Format date range string
        start_str = start_date.strftime(time_format)
        end_str   = end_date.strftime(time_format)
        date_range = f"{start_str} : {end_str}"
        
        # Compute changes
        raw_change = end_price - start_price
        pct_change = (raw_change / start_price) * 100
        log_return = np.log(end_price / start_price) * 100  # now %
        
        rows.append({
            "date_range": date_range,
            "start_date": start_date,
            "end_date": end_date,
            "raw_change": raw_change,
            "pct_change": pct_change,
            "log_return": log_return
        })
    
    df = pd.DataFrame(rows)
    
    # Cumulative compounded % return
    df["cum_return"] = (1 + df["pct_change"]/100).cumprod() - 1
    df["cum_return"] = df["cum_return"] * 100  # %
    
    # Cumulative additive log return (in %)
    df["cum_log_return"] = df["log_return"].cumsum()
    
    # Convert cumulative log return into implied compounded return (%)
    df["cum_return_from_logs"] = (np.exp(df["cum_log_return"]/100) - 1) * 100
    
    return df

In [ ]:
import pandas as pd

def slices_to_dataframe(slices):
    """
    Converts slice output into a DataFrame with raw % change.
    
    Parameters
    ----------
    slices : list of dicts
        Output from fetch_random_slices or fetch_consecutive_slices.
    
    Returns
    -------
    pd.DataFrame
        Columns: [start_date, end_date, start_price, end_price, pct_change]
    """
    if not slices:
        return pd.DataFrame(columns=["start_date", "end_date", "start_price", "end_price", "pct_change"])
    
    df = pd.DataFrame(slices)
    df["pct_change"] = (df["end_price"] - df["start_price"]) / df["start_price"] * 100
    return df


In [ ]:
consecutive_slices = fetch_consecutive_slices(
    symbol, 
    frontier_date, 
    ending_date,
    interval=Resolution.Minute,
    slice_length=timedelta(days=3),
    max_slices = 45
)

consecutive_slices_df = slices_to_extended_dataframe(consecutive_slices)
print(df.head())

In [ ]:
#Testing for Normality
import matplotlib.pyplot as plt
import scipy.stats as stats

def test_normality(df, column="log_return"):
    """
    Tests normality of a return column using Q-Q plot and Shapiro-Wilk test.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame from slices_to_extended_dataframe.
    column : str
        Column to test ("pct_change" or "log_return").
    """
    data = df[column].dropna()
    
    # --- Q-Q Plot ---
    plt.figure(figsize=(6,6))
    stats.probplot(data, dist="norm", plot=plt)
    plt.title(f"Q-Q Plot of {column}", fontsize=14)
    plt.xlabel("Theoretical Quantiles")
    plt.ylabel("Sample Quantiles")
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # --- Shapiro-Wilk Test ---
    stat, p = stats.shapiro(data)
    
    print(f"Shapiro-Wilk Test for {column}")
    print(f"  Test Statistic = {stat:.4f}")
    print(f"  p-value        = {p:.4f}")
    
    if p > 0.05:
        print("✅ Fail to reject H0: Data looks normally distributed.")
    else:
        print("❌ Reject H0: Data does not look normally distributed.")

In [ ]:
#Normality Testing
df = slices_to_extended_dataframe(slices)

# Test for log returns
test_normality(df, column="log_return")

# Or test for simple % changes
test_normality(df, column="pct_change")

In [ ]:
random_slices = fetch_random_slices(
    symbol, frontier_date, ending_date,
    interval=Resolution.Minute,
    slice_length=timedelta(days=3),   # 3-day window
    n_slices=100
)

random_slice_df = slices_to_extended_dataframe(random_slices)

# Test for log returns
test_normality(random_slice_df, column="log_return")

# Or test for simple % changes
test_normality(random_slice_df, column="pct_change")

In [ ]:
#Step 1: Normality + Features (same as before)
from scipy.stats import shapiro, skew, kurtosis
import numpy as np

def extract_features(df, column="log_return"):
    data = df[column].dropna()
    if len(data) < 3:
        return None, 0
    
    features = {
        "mean": np.mean(data),
        "std": np.std(data),
        "skewness": skew(data),
        "kurtosis": kurtosis(data),
        "n_obs": len(data),
    }
    _, p = shapiro(data)
    return features, p


In [ ]:
#Bayesian Optimization Loop
import pandas as pd
import random
from sklearn.ensemble import RandomForestRegressor

def optimize_normality(symbol, frontier_date, ending_date, 
                       n_iter=30, init_points=5,
                       slice_length_bounds=(1, 20),
                       n_slices_bounds=(20, 500),
                       column="log_return"):
    """
    Closed-loop optimizer to maximize normality score (Shapiro-Wilk p-value).
    """
    history = []  # store results
    
    # Step 1: initial random exploration
    for _ in range(init_points):
        days = random.randint(*slice_length_bounds)
        n_slices = random.randint(*n_slices_bounds)
        
        random_slices = fetch_random_slices(
            symbol, frontier_date, ending_date,
            interval=Resolution.Minute,
            slice_length=timedelta(days=days),
            n_slices=n_slices
        )
        df = slices_to_extended_dataframe(random_slices)
        features, score = extract_features(df, column=column)
        
        if features:
            features["slice_length_days"] = days
            features["n_slices"] = n_slices
            features["score"] = score
            history.append(features)
    
    # Step 2: adaptive loop
    for it in range(init_points, n_iter):
        hist_df = pd.DataFrame(history)
        X = hist_df.drop(columns=["score"])
        y = hist_df["score"]
        
        # Train surrogate model
        model = RandomForestRegressor(n_estimators=200, random_state=42)
        model.fit(X, y)
        
        # Generate candidate parameter grid
        candidates = pd.DataFrame([
            {"slice_length_days": d, "n_slices": n, 
             "mean": 0, "std": 0, "skewness": 0, "kurtosis": 0, "n_obs": 0}
            for d in range(slice_length_bounds[0], slice_length_bounds[1]+1)
            for n in range(n_slices_bounds[0], n_slices_bounds[1]+1, 20)
        ])
        
        # Predict scores
        candidates["pred_score"] = model.predict(candidates)
        best = candidates.sort_values("pred_score", ascending=False).iloc[0]
        
        # Evaluate best candidate
        days = int(best["slice_length_days"])
        n_slices = int(best["n_slices"])
        
        random_slices = fetch_random_slices(
            symbol, frontier_date, ending_date,
            interval=Resolution.Minute,
            slice_length=timedelta(days=days),
            n_slices=n_slices
        )
        df = slices_to_extended_dataframe(random_slices)
        features, score = extract_features(df, column=column)
        
        if features:
            features["slice_length_days"] = days
            features["n_slices"] = n_slices
            features["score"] = score
            history.append(features)
        
        print(f"Iteration {it+1}/{n_iter} → days={days}, n_slices={n_slices}, score={score:.4f}")
    
    return pd.DataFrame(history)


In [ ]:
#Run the Optimizer
import pandas as pd
import random
import numpy as np
from datetime import timedelta
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from scipy.stats import shapiro, skew, kurtosis

def extract_features(df, column="log_return"):
    data = df[column].dropna()
    if len(data) < 3:
        return None, 0
    features = {
        "mean": np.mean(data),
        "std": np.std(data),
        "skewness": skew(data),
        "kurtosis": kurtosis(data),
        "n_obs": len(data),
    }
    _, p = shapiro(data)
    return features, p

def optimize_normality(symbol, frontier_date, ending_date, 
                       n_iter=30, init_points=5,
                       slice_length_bounds=(1, 20),
                       n_slices_bounds=(20, 500),
                       column="log_return"):
    """
    Closed-loop optimizer to maximize normality score (Shapiro-Wilk p-value).
    """
    history = []
    
    # Step 1: Initial random exploration
    for _ in range(init_points):
        days = random.randint(*slice_length_bounds)
        n_slices = random.randint(*n_slices_bounds)
        
        random_slices = fetch_random_slices(
            symbol, frontier_date, ending_date,
            interval=Resolution.Minute,
            slice_length=timedelta(days=days),
            n_slices=n_slices
        )
        df = slices_to_extended_dataframe(random_slices)
        features, score = extract_features(df, column=column)
        
        if features:
            features["slice_length_days"] = days
            features["n_slices"] = n_slices
            features["score"] = score
            history.append(features)
    
    # Step 2: Adaptive loop
    for it in range(init_points, n_iter):
        hist_df = pd.DataFrame(history)
        X = hist_df.drop(columns=["score"])
        y = hist_df["score"]
        feature_cols = X.columns.tolist()
        
        model = RandomForestRegressor(n_estimators=200, random_state=42)
        model.fit(X, y)
        
        # Build candidates grid
        candidates = pd.DataFrame([
            {"slice_length_days": d, "n_slices": n,
             "mean": 0, "std": 0, "skewness": 0, "kurtosis": 0, "n_obs": 0}
            for d in range(slice_length_bounds[0], slice_length_bounds[1] + 1)
            for n in range(n_slices_bounds[0], n_slices_bounds[1] + 1, 20)
        ])
        
        # Ensure same column order
        candidates = candidates[feature_cols]
        
        candidates["pred_score"] = model.predict(candidates)
        best = candidates.sort_values("pred_score", ascending=False).iloc[0]
        
        # Evaluate best candidate
        days = int(best["slice_length_days"])
        n_slices = int(best["n_slices"])
        
        random_slices = fetch_random_slices(
            symbol, frontier_date, ending_date,
            interval=Resolution.Minute,
            slice_length=timedelta(days=days),
            n_slices=n_slices
        )
        df = slices_to_extended_dataframe(random_slices)
        features, score = extract_features(df, column=column)
        
        if features:
            features["slice_length_days"] = days
            features["n_slices"] = n_slices
            features["score"] = score
            history.append(features)
        
        print(f"Iteration {it+1}/{n_iter} → days={days}, n_slices={n_slices}, score={score:.4f}")
    
    results = pd.DataFrame(history)
    return results

def plot_results(results):
    """
    Heatmap of normality scores across slice_length_days and n_slices.
    """
    pivot = results.pivot_table(
        index="slice_length_days", columns="n_slices", values="score", aggfunc="mean"
    )
    plt.figure(figsize=(10, 6))
    plt.title("Normality Score Heatmap", fontsize=14)
    plt.xlabel("n_slices")
    plt.ylabel("slice_length_days")
    plt.imshow(pivot, aspect="auto", cmap="viridis", origin="lower")
    plt.colorbar(label="Shapiro-Wilk p-value (Normality Score)")
    plt.xticks(range(len(pivot.columns)), pivot.columns, rotation=45)
    plt.yticks(range(len(pivot.index)), pivot.index)
    plt.show()


In [ ]:
results_df = optimize_normality(
    symbol, frontier_date, ending_date,
    n_iter=20, init_points=5,
    slice_length_bounds=(1, 15),
    n_slices_bounds=(20, 200),
    column="log_return"
)

print("Best run:")
print(results_df.sort_values("score", ascending=False).head(1))

plot_results(results_df)

In [ ]:
#Optimizer 2
import pandas as pd
import numpy as np
import random
from datetime import timedelta
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import shapiro, skew, kurtosis
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# --- Feature extraction ---
def extract_features(df, column="log_return"):
    data = df[column].dropna()
    if len(data) < 3:
        return None, 0
    features = {
        "mean": np.mean(data),
        "std": np.std(data),
        "skewness": skew(data),
        "kurtosis": kurtosis(data),
        "n_obs": len(data),
    }
    _, p = shapiro(data)
    return features, p

# --- Optimizer ---
def optimize_normality(symbol, frontier_date, ending_date, 
                       n_iter=50, init_points=10, batch_size=3,
                       slice_length_bounds=(1, 20),
                       n_slices_bounds=(20, 500),
                       column="log_return"):
    """
    Closed-loop optimizer to maximize Shapiro-Wilk p-value (normality).
    """
    history = []
    
    # Step 1: Initial exploration (random)
    for _ in range(init_points):
        days = random.randint(*slice_length_bounds)
        n_slices = random.randint(*n_slices_bounds)
        random_slices = fetch_random_slices(
            symbol, frontier_date, ending_date,
            interval=Resolution.Minute,
            slice_length=timedelta(days=days),
            n_slices=n_slices
        )
        df = slices_to_extended_dataframe(random_slices)
        features, score = extract_features(df, column=column)
        if features:
            features["slice_length_days"] = days
            features["n_slices"] = n_slices
            features["score"] = score
            history.append(features)
    
    # Step 2: Adaptive search
    for it in range(init_points, n_iter):
        hist_df = pd.DataFrame(history)
        X = hist_df.drop(columns=["score"])
        y = hist_df["score"]
        feature_cols = X.columns.tolist()
        
        model = RandomForestRegressor(n_estimators=300, random_state=42)
        model.fit(X, y)
        
        # Build candidate grid
        candidates = pd.DataFrame([
            {"slice_length_days": d, "n_slices": n,
             "mean": 0, "std": 0, "skewness": 0, "kurtosis": 0, "n_obs": 0}
            for d in range(slice_length_bounds[0], slice_length_bounds[1] + 1)
            for n in range(n_slices_bounds[0], n_slices_bounds[1] + 1, 20)
        ])
        candidates = candidates[feature_cols]
        candidates["pred_score"] = model.predict(candidates)
        
        # Take top-k candidates for evaluation
        topk = candidates.sort_values("pred_score", ascending=False).head(batch_size)
        
        for _, row in topk.iterrows():
            days = int(row["slice_length_days"])
            n_slices = int(row["n_slices"])
            random_slices = fetch_random_slices(
                symbol, frontier_date, ending_date,
                interval=Resolution.Minute,
                slice_length=timedelta(days=days),
                n_slices=n_slices
            )
            df = slices_to_extended_dataframe(random_slices)
            features, score = extract_features(df, column=column)
            if features:
                features["slice_length_days"] = days
                features["n_slices"] = n_slices
                features["score"] = score
                history.append(features)
        
        print(f"Iteration {it+1}/{n_iter} complete. Best score so far: {max(h['score'] for h in history):.4f}")
    
    return pd.DataFrame(history)

# --- 3D Surface Plot ---
def plot_surface(results):
    """
    3D surface plot of slice_length_days × n_slices vs normality score.
    """
    pivot = results.pivot_table(
        index="slice_length_days", columns="n_slices", values="score", aggfunc="mean"
    )
    X, Y = np.meshgrid(pivot.columns, pivot.index)
    Z = pivot.values
    
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection="3d")
    surf = ax.plot_surface(X, Y, Z, cmap="viridis", edgecolor="k", alpha=0.8)
    
    ax.set_title("Normality Score Surface", fontsize=14)
    ax.set_xlabel("n_slices", fontsize=12)
    ax.set_ylabel("Slice Length (days)", fontsize=12)
    ax.set_zlabel("Shapiro-Wilk p-value", fontsize=12)
    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, label="p-value")
    
    plt.show()


In [ ]:
results_df = optimize_normality(
    symbol, frontier_date, ending_date,
    n_iter=3, init_points=4, batch_size=1,
    slice_length_bounds=(1, 180),
    n_slices_bounds=(1, 500),
    column="log_return"
)

print("Best run:")
print(results_df.sort_values("score", ascending=False).head(1))

plot_surface(results_df)

In [ ]:
# Optimizer 3
import pandas as pd
import numpy as np
import random
from datetime import timedelta
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import shapiro, skew, kurtosis
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# --- Feature extraction ---
def extract_features(df, column="log_return"):
    data = df[column].dropna()
    if len(data) < 3:
        return None, 0
    features = {
        "mean": np.mean(data),
        "std": np.std(data),
        "skewness": skew(data),
        "kurtosis": kurtosis(data),
        "n_obs": len(data),
    }
    _, p = shapiro(data)
    return features, p

# --- Optimizer ---
def optimize_normality(symbol, frontier_date, ending_date, 
                       n_iter=50, init_points=10, batch_size=3,
                       slice_length_bounds=(1, 20),
                       n_slices_bounds=(20, 500),
                       column="log_return",
                       alpha=0.05):
    """
    Closed-loop optimizer to maximize Shapiro-Wilk p-value (normality).
    Returns:
      - full history of all runs
      - summary stats of non-significant runs (p <= alpha)
    """
    history = []
    
    # Step 1: Initial exploration (random)
    for _ in range(init_points):
        days = random.randint(*slice_length_bounds)
        n_slices = random.randint(*n_slices_bounds)
        random_slices = fetch_random_slices(
            symbol, frontier_date, ending_date,
            interval=Resolution.Minute,
            slice_length=timedelta(days=days),
            n_slices=n_slices
        )
        df = slices_to_extended_dataframe(random_slices)
        features, score = extract_features(df, column=column)
        if features:
            features["slice_length_days"] = days
            features["n_slices"] = n_slices
            features["score"] = score
            history.append(features)
    
    # Step 2: Adaptive search
    for it in range(init_points, n_iter):
        hist_df = pd.DataFrame(history)
        X = hist_df.drop(columns=["score"])
        y = hist_df["score"]
        feature_cols = X.columns.tolist()
        
        model = RandomForestRegressor(n_estimators=300, random_state=42)
        model.fit(X, y)
        
        # Build candidate grid
        candidates = pd.DataFrame([
            {"slice_length_days": d, "n_slices": n,
             "mean": 0, "std": 0, "skewness": 0, "kurtosis": 0, "n_obs": 0}
            for d in range(slice_length_bounds[0], slice_length_bounds[1] + 1)
            for n in range(n_slices_bounds[0], n_slices_bounds[1] + 1, 20)
        ])
        candidates = candidates[feature_cols]
        candidates["pred_score"] = model.predict(candidates)
        
        # Take top-k candidates for evaluation
        topk = candidates.sort_values("pred_score", ascending=False).head(batch_size)
        
        for _, row in topk.iterrows():
            days = int(row["slice_length_days"])
            n_slices = int(row["n_slices"])
            random_slices = fetch_random_slices(
                symbol, frontier_date, ending_date,
                interval=Resolution.Minute,
                slice_length=timedelta(days=days),
                n_slices=n_slices
            )
            df = slices_to_extended_dataframe(random_slices)
            features, score = extract_features(df, column=column)
            if features:
                features["slice_length_days"] = days
                features["n_slices"] = n_slices
                features["score"] = score
                history.append(features)
        
        best_so_far = max(h['score'] for h in history)
        print(f"Iteration {it+1}/{n_iter} complete. Best score so far: {best_so_far:.4f}")
    
    results = pd.DataFrame(history)
    
    # --- Summarize non-significant runs ---
    non_sig = results[results["score"] <= alpha]
    if non_sig.empty:
        summary = {
            "n_non_significant": 0,
            "mean_p": None,
            "var_p": None,
            "avg_slice_length_days": None,
            "avg_n_slices": None
        }
    else:
        summary = {
            "n_non_significant": len(non_sig),
            "mean_p": non_sig["score"].mean(),
            "var_p": non_sig["score"].var(),
            "avg_slice_length_days": non_sig["slice_length_days"].mean(),
            "avg_n_slices": non_sig["n_slices"].mean()
        }
    
    return results, summary

# --- 3D Surface Plot ---
def plot_surface_with_points(results):
    """
    3D surface plot of slice_length_days × n_slices vs normality score,
    with scatter points for all actual runs.
    """
    # Build pivot for surface
    pivot = results.pivot_table(
        index="slice_length_days", columns="n_slices", values="score", aggfunc="mean"
    )
    X, Y = np.meshgrid(pivot.columns, pivot.index)
    Z = pivot.values
    
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection="3d")
    
    # Surface (smoothed averages)
    surf = ax.plot_surface(X, Y, Z, cmap="viridis", edgecolor="k", alpha=0.6)
    
    # Scatter points (actual trials)
    sc = ax.scatter(
        results["n_slices"], 
        results["slice_length_days"], 
        results["score"], 
        c=results["score"], 
        cmap="viridis", 
        s=40, 
        edgecolor="k"
    )
    
    # Labels
    ax.set_title("Normality Score Surface with Trial Points", fontsize=14)
    ax.set_xlabel("n_slices", fontsize=12)
    ax.set_ylabel("Slice Length (days)", fontsize=12)
    ax.set_zlabel("Shapiro-Wilk p-value", fontsize=12)
    
    # Colorbar
    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, label="p-value")
    
    plt.show()

def plot_surface_with_points_and_path(results):
    """
    3D surface plot of slice_length_days × n_slices vs normality score,
    with scatter points for all actual runs and a line showing the optimizer's search path.
    """
    # Build pivot for surface
    pivot = results.pivot_table(
        index="slice_length_days", columns="n_slices", values="score", aggfunc="mean"
    )
    X, Y = np.meshgrid(pivot.columns, pivot.index)
    Z = pivot.values
    
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection="3d")
    
    # Surface (smoothed averages)
    surf = ax.plot_surface(X, Y, Z, cmap="viridis", edgecolor="k", alpha=0.5)
    
    # Scatter points (actual trials)
    sc = ax.scatter(
        results["n_slices"], 
        results["slice_length_days"], 
        results["score"], 
        c=results["score"], 
        cmap="viridis", 
        s=40, 
        edgecolor="k"
    )
    
    # Trajectory line (optimizer's path, in trial order)
    ax.plot(
        results["n_slices"], 
        results["slice_length_days"], 
        results["score"], 
        color="red", 
        linewidth=2, 
        alpha=0.8, 
        marker="o", 
        markersize=3
    )
    
    # Labels
    ax.set_title("Normality Score Surface with Trials & Search Path", fontsize=14)
    ax.set_xlabel("n_slices", fontsize=12)
    ax.set_ylabel("Slice Length (days)", fontsize=12)
    ax.set_zlabel("Shapiro-Wilk p-value", fontsize=12)
    
    # Colorbar
    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, label="p-value")
    
    plt.show()

In [ ]:
results_df, summary = optimize_normality(
    symbol, frontier_date, ending_date,
    n_iter=50, init_points=10, batch_size=3,
    slice_length_bounds=(1, 15),
    n_slices_bounds=(20, 200),
    column="log_return",
    alpha=0.05
)

print("Full results (first few):")
print(results_df.head())

print("\nSummary of Non-Significant Runs:")
print(summary)

plot_surface(results_df)

plot_surface_with_points_and_path(results_df)